In [18]:
# Aggregated Transaction code
from typing import List
from typing import Any
from dataclasses import dataclass
import json
import os
import mysql.connector
_connection = None

def connect_to_mysql():
    global _connection
    if not _connection:
        _connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password=""
        )

def create_database():
    mycursor = _connection.cursor(buffered=True)
    mycursor.execute("CREATE DATABASE IF NOT EXISTS PhonPe_Project")
    mycursor.execute("USE Youtube_project") 

def create_map_transaction():
    mycursor = _connection.cursor(buffered=True)
    mycursor.execute('''CREATE TABLE IF NOT EXISTS PhonPe_Project.Aggregated_transaction (
             Type    varchar(255),
             State   varchar(255),
             Payment_type varchar(255),
             Year  YEAR,
             Quarter_value int,
             Transaction_count  BIGINT,
             Transaction_amount BIGINT
             )''')
     
@dataclass
class PaymentInstrument:
    type: str
    count: int
    amount: float

    @staticmethod
    def from_dict(obj: Any) -> 'PaymentInstrument':
        _type = str(obj.get("type"))
        _count = int(obj.get("count"))
        _amount = float(obj.get("amount"))
        return PaymentInstrument(_type, _count, _amount)

@dataclass
class TransactionData:
    name: str
    paymentInstruments: List[PaymentInstrument]

    @staticmethod
    def from_dict(obj: Any) -> 'TransactionData':
        _name = str(obj.get("name"))
        _payment_instruments = [PaymentInstrument.from_dict(y) for y in obj.get("paymentInstruments")]
        return TransactionData(_name, _payment_instruments)

@dataclass
class Data:
    transactionData: List[TransactionData]

    @staticmethod
    def from_dict(obj: Any) -> 'Data':
        _transaction_data = [TransactionData.from_dict(y) for y in obj.get("transactionData")]
        return Data(_transaction_data)

@dataclass
class Root:
    success: bool
    code: str
    data: Data
    responseTimestamp: float

    @staticmethod
    def from_dict(obj: Any) -> 'Root':
        _success = str(obj.get("success"))
        _code = str(obj.get("code"))
        _data = Data.from_dict(obj.get("data"))
        _responseTimestamp = float(obj.get("responseTimestamp"))
        return Root(_success, _code, _data, _responseTimestamp)

def format_state_name(state_name):
    if state_name== "andaman-&-nicobar-islands":
        return "Andaman & Nicobar"
    elif state_name=="dadra-&-nagar-haveli-&-daman-&-diu":
        return "Dadra and Nagar Haveli and Daman and Diu"
    else:
        words = state_name.split("-")
        formatted_words = [word.capitalize() for word in words]
        formatted_state_name = " ".join(formatted_words)
        return formatted_state_name

def Map_district_data(Type,State_data, Name_data, year_value, Quarter_value, Count_data, Amount_data):
    sql = "INSERT INTO phonPe_project.Aggregated_transaction (Type,State, Payment_type, Year, Quarter_value, Transaction_count, Transaction_amount) VALUES (%s,%s, %s, %s, %s, %s, %s)"
    val = (Type,State_data, Name_data, year_value, Quarter_value, Count_data, Amount_data)
    cursor = _connection.cursor(buffered=True)
    cursor.execute(sql, val)

def data_extraction():
    path = "C:/Users/BARATH KANNAN/OneDrive/Desktop/Project_phone_pe/DATA/aggregated/transaction/country/india/state/"
    Agg_state_list = os.listdir(path)
    Agg_state_list
    Type="Transaction"
    for i in Agg_state_list:
        State_data = format_state_name(i)
        yr_data = path + i + "/"
        year_data = os.listdir(yr_data)
        for k in year_data:
            year_value = k
            json_data = yr_data + k + "/"
            Quarter_value=0
            for filename in os.listdir(json_data):
                Quarter_value= Quarter_value+1
                f = os.path.join(json_data, filename)
                if os.path.isfile(f):
                    with open(f) as fd:
                        jsonstring = json.loads(fd.read())
                        root = Root.from_dict(jsonstring)
                        for transaction_data in root.data.transactionData:
                            Name_data = transaction_data.name
                            for payment_instrument in transaction_data.paymentInstruments:
                                Count_data = payment_instrument.count
                                Amount_data = round(payment_instrument.amount, 2)
                                Map_district_data(Type,State_data, Name_data, year_value,Quarter_value, Count_data, Amount_data)

connect_to_mysql()
create_database()   
create_map_transaction()  
data_extraction()
_connection.commit()
print("created")

created


In [19]:
#Aggregate USER
from typing import List
from typing import Any
from dataclasses import dataclass
import json
import mysql.connector
import os

_connection = None

def connect_to_mysql():
    global _connection
    if not _connection:
        _connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password=""
        )
    return _connection

def create_database():
    connection = connect_to_mysql()
    mycursor = connection.cursor(buffered=True)
    mycursor.execute("CREATE DATABASE IF NOT EXISTS PhonPe_Project")
    mycursor.execute("USE PhonPe_Project")
    mycursor.close()

def create_map_transaction():
    connection = connect_to_mysql()
    mycursor = connection.cursor(buffered=True)
    mycursor.execute('''CREATE TABLE IF NOT EXISTS PhonPe_Project.Aggregated_User (
             Type  varchar(255),
             State   varchar(255),
             year YEAR,
             Quarter_value int,
             registered_user  BIGINT,
             mobile_brand  varchar(255),
             User_Count BIGINT,
             User_Percentage float
             )''')
    mycursor.close()



def format_state_name(state_name):
    if state_name== "andaman-&-nicobar-islands":
        return "Andaman & Nicobar"
    elif state_name=="dadra-&-nagar-haveli-&-daman-&-diu":
        return "Dadra and Nagar Haveli and Daman and Diu"
    else:
        words = state_name.split("-")
        formatted_words = [word.capitalize() for word in words]
        formatted_state_name = " ".join(formatted_words)
        return formatted_state_name

def Map_district_data(Type,user_state_data, user_year_value, Quarter_value, registered_users, Brand_name, No_of_Users, Percentage_of_Users):
    connect_to_mysql()  # Ensure connection to MySQL
    sql = "INSERT INTO PhonPe_Project.Aggregated_User (Type,State, year, Quarter_value, registered_user, mobile_brand, User_Count, User_Percentage) VALUES (%s, %s, %s,%s, %s, %s, %s, %s)"
    val = (Type,user_state_data, user_year_value,Quarter_value, registered_users, Brand_name, No_of_Users, Percentage_of_Users)
    cursor = _connection.cursor(buffered=True)
    cursor.execute(sql, val)

@dataclass
class DeviceData:
    brand: str
    count: int
    percentage: float

    @staticmethod
    def from_dict(obj: Any) -> 'DeviceData':
        brand = str(obj.get("brand"))
        count = int(obj.get("count"))
        percentage = float(obj.get("percentage"))
        return DeviceData(brand, count, percentage)

@dataclass
class Data:
    aggregated: Any
    usersByDevice: List[DeviceData]

    @staticmethod
    def from_dict(obj: Any) -> 'Data':
        aggregated = obj.get("aggregated")
        users_by_device = []
        if obj.get("usersByDevice"):
            users_by_device = [DeviceData.from_dict(item) for item in obj.get("usersByDevice")]
        return Data(aggregated, users_by_device)

@dataclass
class Root:
    success: bool
    code: str
    data: Data
    responseTimestamp: float

    @staticmethod
    def from_dict(obj: Any) -> 'Root':
        success = obj.get("success")
        code = obj.get("code")
        data = Data.from_dict(obj.get("data"))
        response_timestamp = obj.get("responseTimestamp")
        return Root(success, code, data, response_timestamp)
    
def data_extraction():   
        json_data = "C:/Users/BARATH KANNAN/OneDrive/Desktop/Project_phone_pe/DATA/aggregated/user/country/india/state/"
        Agg_user_list = os.listdir(json_data)
        Type="User"
        for i in Agg_user_list:
            user_state_data = format_state_name(i)
            user_year_data = json_data + i + "/"
            user_yr_data = os.listdir(user_year_data)
            for k in user_yr_data:
                user_year_value = k
                json_user_data = user_year_data + k + "/"
                Quarter_value=0
                for filename in os.listdir(json_user_data):
                    Quarter_value= Quarter_value+1
                    f = os.path.join(json_user_data, filename)
                    with open(f) as fd:
                        jsonstring = json.loads(fd.read())
                        root = Root.from_dict(jsonstring)
                        registered_users = root.data.aggregated.get("registeredUsers")
                        for j in root.data.usersByDevice:
                            Brand_name = j.brand
                            No_of_Users = j.count
                            Percentage_of_Users = j.percentage
                            Map_district_data(Type, user_state_data, user_year_value, Quarter_value, registered_users, Brand_name, No_of_Users, Percentage_of_Users)

                            
connect_to_mysql()
create_database()
create_map_transaction()
data_extraction()
_connection.commit()
print("completed")

completed


In [20]:
# MAP- Transaction Data
from typing import List
from typing import Any
from dataclasses import dataclass
import json
import os
import mysql.connector
_connection = None

def connect_to_mysql():
    global _connection
    if not _connection:
        _connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password=""
        )

def create_database():
     mycursor=_connection.cursor(buffered=True)
     mycursor.execute("CREATE DATABASE IF NOT EXISTS PhonPe_Project")
     mycursor.execute("USE Youtube_project") 

def create_map_transaction():
    mycursor=_connection.cursor(buffered=True)
    mycursor.execute('''CREATE TABLE IF NOT EXISTS PhonPe_Project.map_transaction (
             Type  varchar(255),
             State   varchar(255),
             District varchar(255),
             Year  YEAR,
             Quarter_value  INT,
             transaction_count  BIGINT,
             transaction_amount BIGINT
             )''')
     

@dataclass
class Metric:
    type: str
    count: int
    amount: float

    @staticmethod
    def from_dict(obj: Any) -> 'Metric':
        _type = str(obj.get("type"))
        _count = int(obj.get("count"))
        _amount = float(obj.get("amount"))
        return Metric(_type, _count, _amount)

@dataclass
class HoverDataList:
    name: str
    metric: List[Metric]

    @staticmethod
    def from_dict(obj: Any) -> 'HoverDataList':
        _name = str(obj.get("name"))
        _metric = [Metric.from_dict(y) for y in obj.get("metric")]
        return HoverDataList(_name, _metric)

@dataclass
class Data:
    hoverDataList: List[HoverDataList]

    @staticmethod
    def from_dict(obj: Any) -> 'Data':
        _hoverDataList = [HoverDataList.from_dict(y) for y in obj.get("hoverDataList")]
        return Data(_hoverDataList)

@dataclass
class Root:
    success: bool
    code: str
    data: Data
    responseTimestamp: float

    @staticmethod
    def from_dict(obj: Any) -> 'Root':
        _success = str(obj.get("success"))
        _code = str(obj.get("code"))
        _data = Data.from_dict(obj.get("data"))
        _responseTimestamp = float(obj.get("responseTimestamp"))
        return Root(_success, _code, _data, _responseTimestamp)

def format_state_name(state_name):
    if state_name== "andaman-&-nicobar-islands":
        return "Andaman & Nicobar"
    elif state_name=="dadra-&-nagar-haveli-&-daman-&-diu":
        return "Dadra and Nagar Haveli and Daman and Diu"
    else:
        words = state_name.split("-")
        formatted_words = [word.capitalize() for word in words]
        formatted_state_name = " ".join(formatted_words)
        return formatted_state_name

def Map_district_data(Type, State_data, Name_data,year_value, Quarter_value, Count_data, Amount_data):
    sql = "INSERT INTO phonPe_project.map_transaction (Type, State, District, Year, Quarter_value, transaction_count, transaction_amount) VALUES (%s,%s, %s, %s, %s, %s, %s)"
    val = (Type, State_data,Name_data, year_value, Quarter_value, Count_data, Amount_data)
    cursor = _connection.cursor(buffered=True)
    cursor.execute(sql, val)


def data_extraction():
    path="C:/Users/BARATH KANNAN/OneDrive/Desktop/Project_phone_pe/DATA/map/transaction/hover/country/india/state/"
    Agg_state_list=os.listdir(path)
    Agg_state_list
    Type ="Transaction"
    for i in Agg_state_list:
        State_data=format_state_name(i)
        yr_data=path+i+"/"
        year_data=os.listdir(yr_data)
        for k in year_data:
            year_value=k
            json_data= yr_data+k+"/"
            Quarter_value=0
            for filename in os.listdir(json_data):
                Quarter_value= Quarter_value+1
                f = os.path.join(json_data, filename)
                if os.path.isfile(f):
                    with open(f) as fd:
                        jsonstring = json.loads(fd.read())
                        root = Root.from_dict(jsonstring)
                        for y in (root.data.hoverDataList):
                            Name_data=y.name
                            for x in y.metric:
                                Count_data=x.count
                                Amount_data=round(x.amount,2)
                                Map_district_data(Type, State_data,Name_data,year_value, Quarter_value, Count_data, Amount_data)


connect_to_mysql()
create_database()   
create_map_transaction()  
data_extraction()
_connection.commit()
print("created")



created


In [21]:
#map_user
from typing import List
from typing import Any
from dataclasses import dataclass
import json
import mysql.connector
import os

_connection = None

def connect_to_mysql():
    global _connection
    if not _connection:
        _connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password=""
        )

def create_database():
    mycursor = _connection.cursor(buffered=True)
    mycursor.execute("CREATE DATABASE IF NOT EXISTS PhonPe_Project")
    mycursor.execute("USE PhonPe_Project")

def create_map_transaction():
    mycursor = _connection.cursor(buffered=True)
    mycursor.execute('''CREATE TABLE IF NOT EXISTS PhonPe_Project.map_User (
             Type  varchar(255),
             State   varchar(255),
             Year  YEAR,
             Quarter_value  int,
             district_name varchar(255),
             registered_user  BIGINT,
             App_Open_count  BIGINT
             )''')

def format_state_name(state_name):
    if state_name== "andaman-&-nicobar-islands":
        return "Andaman & Nicobar"
    elif state_name=="dadra-&-nagar-haveli-&-daman-&-diu":
        return "Dadra and Nagar Haveli and Daman and Diu"
    else:
        words = state_name.split("-")
        formatted_words = [word.capitalize() for word in words]
        formatted_state_name = " ".join(formatted_words)
        return formatted_state_name

def Map_district_data(Type, user_state_data, user_year_value, Quarter_value, district_name, registered_users,App_Opens):
    connect_to_mysql()  # Ensure connection to MySQL
    sql = "INSERT INTO PhonPe_Project.map_User (Type, State, Year, Quarter_value, district_name, registered_user, App_Open_count) VALUES (%s,%s, %s,%s, %s, %s, %s)"
    val = (Type,user_state_data, user_year_value, Quarter_value, district_name, registered_users,App_Opens)
    cursor = _connection.cursor(buffered=True)
    cursor.execute(sql, val)

@dataclass
class HoverData:
    registeredUsers: int
    appOpens: int

    @staticmethod
    def from_dict(obj: Any) -> 'HoverData':
        registered_users = int(obj.get("registeredUsers"))
        app_opens = int(obj.get("appOpens"))
        return HoverData(registered_users, app_opens)

@dataclass
class HoverDataList:
    name: str
    hoverData: HoverData

    @staticmethod
    def from_dict(obj: Any) -> 'HoverDataList':
        _name = str(obj.get("name"))
        _hover_data = HoverData.from_dict(obj.get("hoverData"))
        return HoverDataList(_name, _hover_data)

@dataclass
class Data:
    hoverDataList: List[HoverDataList]

    @staticmethod
    def from_dict(obj: Any) -> 'Data':
        hover_data_list = []
        for key, value in obj.items():
            hover_data_list.append(HoverDataList(key, HoverData.from_dict(value)))
        return Data(hover_data_list)

@dataclass
class Root:
    success: bool
    code: str
    data: Data
    responseTimestamp: float

    @staticmethod
    def from_dict(obj: Any) -> 'Root':
        _success = bool(obj.get("success"))
        _code = str(obj.get("code"))
        _data = Data.from_dict(obj.get("data").get("hoverData"))
        _responseTimestamp = float(obj.get("responseTimestamp"))
        return Root(_success, _code, _data, _responseTimestamp)

def data_extraction():
        json_data = "C:/Users/BARATH KANNAN/OneDrive/Desktop/Project_phone_pe/DATA/map/user/hover/country/india/state/"
        map_user_list = os.listdir(json_data)
        Type="User"
        for i in map_user_list:
            user_state_data = format_state_name(i)
            user_year_data = json_data + i + "/"
            user_yr_data = os.listdir(user_year_data)
            for k in user_yr_data:
                user_year_value = k
                json_user_data = user_year_data + k + "/"
                Quarter_value=0
                for filename in os.listdir(json_user_data):
                    Quarter_value= Quarter_value+1
                    f = os.path.join(json_user_data, filename)
                    with open(f) as fd:
                        jsonstring = json.loads(fd.read())
                        root = Root.from_dict(jsonstring)
                        hover_data = root.data.hoverDataList
                        for hover_data_list in hover_data:
                            district_name = hover_data_list.name
                            registered_users = hover_data_list.hoverData.registeredUsers
                            App_Opens = hover_data_list.hoverData.appOpens
                            Map_district_data(Type,user_state_data, user_year_value, Quarter_value, district_name, registered_users,App_Opens)


connect_to_mysql()
create_database()
create_map_transaction()
data_extraction()

_connection.commit()
print("created")

created


In [22]:
#TOP USER
from typing import List
from typing import Any
from dataclasses import dataclass
import json
import mysql.connector
import os

_connection = None

def connect_to_mysql():
    global _connection
    if not _connection:
        _connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password=""
        )

def create_database(): 
    mycursor = _connection.cursor(buffered=True)
    mycursor.execute("CREATE DATABASE IF NOT EXISTS PhonPe_Project")
    mycursor.execute("USE PhonPe_Project")

def create_map_transaction():
    
    mycursor = _connection.cursor(buffered=True)
    mycursor.execute('''CREATE TABLE IF NOT EXISTS PhonPe_Project.top_User (
             Type varchar(255),
             State   varchar(255),
             year YEAR,
             Quarter int,
             pincode int,
             Registered_user int
             )''')
    

def format_state_name(state_name):
    if state_name== "andaman-&-nicobar-islands":
        return "Andaman & Nicobar"
    elif state_name=="dadra-&-nagar-haveli-&-daman-&-diu":
        return "Dadra and Nagar Haveli and Daman and Diu"
    else:
        words = state_name.split("-")
        formatted_words = [word.capitalize() for word in words]
        formatted_state_name = " ".join(formatted_words)
        return formatted_state_name

def Map_district_data(Type, user_state_data, user_year_value, Quarter_value, Pincode, Registered_users):
    connect_to_mysql()  
    sql = "INSERT INTO PhonPe_Project.top_User (Type, State, year, Quarter, pincode, registered_user) VALUES (%s, %s, %s,%s, %s, %s)"
    val = (Type, user_state_data, user_year_value, Quarter_value, Pincode, Registered_users)
    cursor = _connection.cursor(buffered=True)
    cursor.execute(sql, val)
    
@dataclass
class PincodeData:
    name: str
    registeredUsers: int

    @staticmethod
    def from_dict(obj: Any) -> 'PincodeData':
        name = str(obj.get("name"))
        registered_users = int(obj.get("registeredUsers"))
        return PincodeData(name, registered_users)

@dataclass
class Data:
    districts: List[PincodeData]
    pincodes: List[PincodeData]

    @staticmethod
    def from_dict(obj: Any) -> 'Data':
        districts = []
        if obj.get("districts"):
            districts = [PincodeData.from_dict(item) for item in obj.get("districts")]
        pincodes = [PincodeData.from_dict(item) for item in obj.get("pincodes")]
        return Data(districts, pincodes)

@dataclass
class Root:
    success: bool
    code: str
    data: Data
    responseTimestamp: float

    @staticmethod
    def from_dict(obj: Any) -> 'Root':
        success = obj.get("success")
        code = obj.get("code")
        data = Data.from_dict(obj.get("data"))
        response_timestamp = obj.get("responseTimestamp")
        return Root(success, code, data, response_timestamp)
    

def data_extraction():
        json_data_dir = "C:/Users/BARATH KANNAN/OneDrive/Desktop/Project_phone_pe/DATA/top/user/country/india/state/" 
        Agg_user_list = os.listdir(json_data_dir)
        Type="User"
        for i in Agg_user_list:
            user_state_data = format_state_name(i)
            user_year_data_dir = json_data_dir + i + "/"
            user_yr_data = os.listdir(user_year_data_dir)
            for k in user_yr_data:
                user_year_value = k
                json_user_data_dir = user_year_data_dir + k + "/"
                Quarter_value=0
                for filename in os.listdir(json_user_data_dir):
                    Quarter_value=Quarter_value+1
                    f = os.path.join(json_user_data_dir, filename)
                    with open(f, 'r') as file:
                        json_data = json.load(file)
                        root_obj = Root.from_dict(json_data)
                        for pincode_data in root_obj.data.pincodes:
                            Pincode= pincode_data.name
                            Registered_users= pincode_data.registeredUsers
                            Map_district_data(Type, user_state_data, user_year_value,Quarter_value, Pincode, Registered_users)
connect_to_mysql() 
create_database()
create_map_transaction()
data_extraction()
_connection.commit()
print("created")

created


In [23]:
# TOP Transaction code
from typing import List
from typing import Any
from dataclasses import dataclass
import json
import os
import mysql.connector

_connection = None

def connect_to_mysql():
    global _connection
    if not _connection:
        _connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password=""
        )

def create_database():
    mycursor = _connection.cursor(buffered=True)
    mycursor.execute("CREATE DATABASE IF NOT EXISTS PhonPe_Project")
    mycursor.execute("USE PhonPe_Project")

def create_map_transaction():
    mycursor = _connection.cursor(buffered=True)
    mycursor.execute('''CREATE TABLE IF NOT EXISTS PhonPe_Project.top_transaction (
             Type  varchar(255),
             State varchar(255),
             location_pincode BIGINT,
             Year YEAR,
             Quarter_value  int,
             transaction_count BIGINT,
             transaction_amount BIGINT
             )''')

@dataclass
class Metric:
    type: str
    count: int
    amount: float

    @staticmethod
    def from_dict(obj: Any) -> 'Metric':
        _type = str(obj.get("type"))
        _count = int(obj.get("count"))
        _amount = float(obj.get("amount"))
        return Metric(_type, _count, _amount)

@dataclass
class District:
    entityName: str
    metric: Metric

    @staticmethod
    def from_dict(obj: Any) -> 'District':
        _entityName = str(obj.get("entityName"))
        _metric = Metric.from_dict(obj.get("metric"))
        return District(_entityName, _metric)

@dataclass
class Pincode:
    entityName: str
    metric: Metric

    @staticmethod
    def from_dict(obj: Any) -> 'Pincode':
        _entityName = str(obj.get("entityName"))
        _metric = Metric.from_dict(obj.get("metric"))
        return Pincode(_entityName, _metric)

@dataclass
class Data:
    districts: List[District]
    pincodes: List[Pincode]

    @staticmethod
    def from_dict(obj: Any) -> 'Data':
        _districts = [District.from_dict(item) for item in obj.get("districts", [])]
        _pincodes = [Pincode.from_dict(item) for item in obj.get("pincodes", [])]
        return Data(_districts, _pincodes)

@dataclass
class Root:
    success: bool
    code: str
    data: Data
    responseTimestamp: float

    @staticmethod
    def from_dict(obj: Any) -> 'Root':
        _success = bool(obj.get("success"))
        _code = str(obj.get("code"))
        _data = Data.from_dict(obj.get("data"))
        _responseTimestamp = float(obj.get("responseTimestamp"))
        return Root(_success, _code, _data, _responseTimestamp)

def format_state_name(state_name):
    if state_name== "andaman-&-nicobar-islands":
        return "Andaman & Nicobar"
    elif state_name=="dadra-&-nagar-haveli-&-daman-&-diu":
        return "Dadra and Nagar Haveli and Daman and Diu"
    else:
        words = state_name.split("-")
        formatted_words = [word.capitalize() for word in words]
        formatted_state_name = " ".join(formatted_words)
        return formatted_state_name

def insert_data_into_mysql(Type, state_data, name_data, year_value, Quarter_value, count_data, amount_data):
    sql = "INSERT INTO PhonPe_Project.top_transaction (Type, State, location_pincode, Year, Quarter_value, transaction_count, transaction_amount) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    val = (Type, state_data, name_data, year_value, Quarter_value, count_data, amount_data)
    cursor = _connection.cursor(buffered=True)
    cursor.execute(sql, val)

def data_extraction():
    path = r"C:/Users/BARATH KANNAN/OneDrive/Desktop/Project_phone_pe/DATA/top/transaction/country/india/state/"  
    Agg_state_list = os.listdir(path)
    Type="Transaction"
    for i in Agg_state_list:
        state_data = format_state_name(i)
        yr_data = os.path.join(path, i)
        year_data = os.listdir(yr_data)
        for k in year_data:
            year_value = k
            json_data = os.path.join(yr_data, k)
            Quarter_value=0
            for filename in os.listdir(json_data):
                Quarter_value=Quarter_value+1
                f = os.path.join(json_data, filename)
                if os.path.isfile(f):
                    with open(f) as fd:
                        json_dict = json.load(fd)
                        root = Root.from_dict(json_dict)
                        for pincode_data in root.data.pincodes:
                            name_data = pincode_data.entityName
                            count_data = pincode_data.metric.count
                            amount_data = round(pincode_data.metric.amount, 2)
                            insert_data_into_mysql(Type,state_data, name_data, year_value, Quarter_value, count_data, amount_data)

connect_to_mysql()
create_database()
create_map_transaction()
data_extraction()
_connection.commit()
_connection.close()
print("Data inserted")



Data inserted
